In [80]:
import numpy as np
from numpy.linalg import eig
import itertools
import functools
import time
import h5py
def norm_complex(arr: np.ndarray):
    if len(arr.shape) > 1:
        ret_out = np.zeros_like(arr)
        for i in range(arr.shape[0]):
            ret_out[i,:] = arr[i]/np.sqrt((np.abs(arr[i])**2).sum())  
        return ret_out
    else:
        return arr/np.sqrt((np.abs(arr)**2).sum())

In [2]:
sx = np.array([[0, 1], [1, 0]])
sy = np.array([[0, 1j], [-1j, 0]])
sz = np.array([[1, 0], [0, -1]])
basis = np.stack([np.eye(2), sx, sy, sz])

n = 4 # Nb of qubits
J = 4**n # Matches the number of bases
I = 6**n # Matches  R*A = 2^n * 3^n = 6^n
d = R = 2**n # matrix dimension and number of possibilities for R^a_s ({-1, 1}^n)
A = 3**n # Number of possible measurements
npa = np.array
b = npa(list(itertools.product(range(4), repeat=n))) # {I, x, y, z}^n
a = npa(list(itertools.product(range(1, 4), repeat=n))) # {x,y,z}^n
# r = npa(list(itertools.product([1, 0], repeat=n))) # {0, 1}^n -> acts like a mask for which bases to select
r = npa(list(itertools.product([-1, 1], repeat=n)))
def projectors(idx_list, r_):
    """
    Returns the P^{a_i}_{s_i} list of projection matrices
    Note1: the evs returned by numpy and the ones obtained don't match, 
    but as they are not unique, it is still correct.
    Note2: because we are working in R^2, we can use the [-1, 1] indexing
    (which works differently in R - it removes the neg col - but still the same here)
    """
    r_idx = [1 if i==-1 else 0 for i in r_]
    evs = [eig(basis[i])[1] for i in idx_list]
    selected_evs = np.array([ev[:, r_idx[i]] for i, ev in enumerate(evs)])
    ret = npa([np.outer(np.conj(ev), ev) for ev in selected_evs])
    # print(ret)
    return ret

# Corresponds to P^a_s in paper (each row here is a matrix), size: 2^n x 3^n flattened
# The projectors matrices 
Pra = []
for j in range(A):
    for i in range(R):
        # print(a[j], r[i])
        # print(projectors(a[j], r[i]))
        # print()
        Pra.append(npa(functools.reduce(np.kron, projectors(a[j], r[i]))).flatten("F"))
Pra = npa(Pra)

In [3]:
with h5py.File('Pra_py.h5', 'w') as file:
    file.create_dataset('data', data=Pra.astype(float).T)

/tmp/ipykernel_14654/1398590864.py:2: ComplexWarning: Casting complex values to real discards the imaginary part
  file.create_dataset('data', data=Pra.astype(float).T)


In [4]:
#### All possible combinations of a (kron product of all permutations)
# Pauli basis for n qubit 
sig_b = npa([functools.reduce(np.kron, (basis[b[i,:], :, :])) for i in range(J)])


# Only used for the calculation of rho_hat, 
### Maybe matches to p_a,s = P(R^a = s) in paper, size: 6^n x 4^n
# For every comb of the bases (j in 0:J), then for every activation of the bases (r[s in S, ])
# Matrix P_{(r,a),b} 
P_rab = np.zeros((I, J))
for j in range(J):
    tmp = np.zeros((R, A))
    for s in range(R): # r_neg[s] = [-1, 1, -1, 1], b[j]=[1, 2, 0, 2], a[l] = [2, 4, 1, 1] 
        for l in range(A): #  r_neg[s, b[j] != 0] = [-1, 1, 1]
            val = np.prod(r[s, b[j] != 0])\
                * np.prod(a[l, b[j] != 0] == b[j, b[j]!=0]) # a[l, b[j] != 0] == b[j, b[j] != 0] <=> [2, 4, 1] != [1, 2, 2] 
            tmp[s,l] = val
    P_rab[:, j] = tmp.flatten(order="F")

In [6]:
# with h5py.File('P_rab_py.h5', 'w') as file:
#     file.create_dataset('data', data=P_rab.T)

In [ ]:
# with h5py.File('sig_b_py.h5', 'w') as file:
#     file.create_dataset('data', data=sig_b.astype(float).reshape(256, 256))

In [46]:
u = norm_complex(np.random.multivariate_normal(np.zeros(d*2),np.eye(d*2)/100, size=(d)).view(np.complex128))
dens_ma = np.conj(u.T) @ u /d
np.linalg.matrix_rank(dens_ma)

16

In [96]:
v1[0:int(d/2)] = 1
v1 = norm_complex(v1)
v2 = np.zeros(d)
v2[int(d/2):d] = i
v2 = norm_complex(v2)
dens_ma = 1/2 * np.outer(v1, np.conj(v1)) + 1/2 * np.outer(v2, np.conj(v2))
w = 0.98
dens_ma = w * dens_ma + (1 - w) * np.eye(d)/d


In [98]:
np.linalg.matrix_rank(dens_ma)

16

In [7]:
# Pure state - rank1
dens_ma = np.zeros((d, d))
dens_ma[0, 0] = 1

# Rank2
v1 = np.zeros(d)
v1[0:int(d/2)] = 1
v1 = norm_complex(v1)
v2 = np.zeros(d)
v2[int(d/2):d] = 1j
v2 = norm_complex(v2)
dens_ma = 1/2 * np.outer(v1, np.conj(v1)) + 1/2 * np.outer(v2, np.conj(v2))

# Approx rank2
v1[0:int(d/2)] = 1
v1 = norm_complex(v1)
v2 = np.zeros(d)
v2[int(d/2):d] = 1j
v2 = norm_complex(v2)
dens_ma = 1/2 * np.outer(v1, np.conj(v1)) + 1/2 * np.outer(v2, np.conj(v2))
w = 0.98
dens_ma = w * dens_ma + (1 - w) * np.eye(d)/d

# Maximal mixed state (rank=16) - ground truth
u = norm_complex(np.random.multivariate_normal(np.zeros(d*2),np.eye(d*2)/100, size=(d)).view(np.complex128))
dens_ma = np.conj(u.T) @ u /d

# Corresponds to Tr(rho \dot P^a_s), which in turn corresponds to p_a,s
# These probabilities are the true ones, so we do not have access to them
# We use them below to measure the system
Prob_ar = np.zeros((A, R))
if n==1:
    for i in range(A):
        for j in range(R):
            Prob_ar[i,j] = dens_ma.flatten(order="F") @ projectors(a[i], r[j])
else:
    for i in range(A):
        for j in range(R):
            Prob_ar[i,j] = np.diag(dens_ma @ npa(functools.reduce(np.kron, projectors(a[i], r[j])))).sum()
Prob_ar = np.real(Prob_ar)


# For each observable, we sample according to the true probabilities calculated above
# n_size samples, and then give the probability
# For example: 
# if n=4 (qubits) and a_i = {x, x, y, z}, then an outcome could be s_j {-1, 1, -1, 1}
# For this pair of a,s, we calculate the number of times we sampled this situation (the H == s part) 
#

# Nb of times we repeat the measurements
n_size = 2000
p_ra = np.zeros((R, A)) # = \hat{p}_a,s
for i, x in enumerate(Prob_ar):
    H = np.random.choice(R, n_size, replace=True, p=x) #n_size elements
    out = []
    for s in range(R):
        out.append((H==s).sum()/n_size) # Calculate the empirical prob of this combination
    p_ra[:, i] = out
# Transform matrix to vector form
p_ra1 = p_ra.flatten(order="F")

temp1 = p_ra1 @ P_rab
temp1 = temp1/d

# Calculate coefs rho_b
rho_b = [0] * J
for i in range(J):
    rho_b[i] = temp1[i]/(3**((b[i] == 0).sum()))

# Calculate density using inversion technique
rho_hat = np.zeros((d, d), dtype=np.complex128)
for s in range(J):
    rho_hat += rho_b[s] * sig_b[s]
# rho_hat = rho_hat.astype(float)
u_hat = eig(rho_hat)[1]

# renormalize lambda_hat
lamb_til = eig(rho_hat)[0]
lamb_til[lamb_til < 0] = 0
lamb_hat = lamb_til/lamb_til.sum()

/tmp/ipykernel_14654/1648047235.py:16: ComplexWarning: Casting complex values to real discards the imaginary part
  Prob_ar[i,j] = np.diag(dens_ma @ npa(functools.reduce(np.kron, projectors(a[i], r[j])))).sum()


In [79]:
import time
a = time.perf_counter()
part1 = 0
part2 = 0
part3 = 0
for _ in range(10):
    for i in range(int(2*d)):
        p1s = time.perf_counter()
        p1 = (Pra_m @ tem_can - p_ra1)**2
        p1e = time.perf_counter()
        # part1 += p1e-p1s
        # p2s = time.perf_counter()
        # p2 = np.repeat(p1[:, np.newaxis], p_ra1.shape[0], axis=1)
        # p2e = time.perf_counter()
        # part2 += p2e-p2s
        # p3s = time.perf_counter()
        # p3 = (p2 - p_ra1[:, np.newaxis])**2 
        # p3e = time.perf_counter()
        # part3 += p3e-p3s
        # ss = (np.repeat((Pra_m @ tem_can)[:, np.newaxis], p_ra1.shape[0], axis=1) - p_ra1[:, np.newaxis])**2
b = time.perf_counter()
print(f"Tot: {b-a}; p1: {part1/(b-a)}; p2: {part2/(b-a)}; p3: {part3/(b-a)}")


Tot: 0.12956043099984527; p1: 0.0; p2: 0.0; p3: 0.0


In [75]:
(Pra_m @ tem_can - p_ra1).shape

(1296,)

In [84]:
rho = np.zeros((d, d))
Te = np.random.standard_exponential(d) # Initial Y_i^0
Id = np.eye(d)
U = u_hat # eigenvectors of \hat(rho) found using inversion, initial V_i^0
Lamb = Te/Te.sum() # gamma^0
U = u_hat # eigenvectors of \hat(rho) found using inversion, initial V_i^0
be = 1
Pra_m = npa(Pra).reshape((I, J))
for i in range(600):
    for j in range(d):
        Te_can = Te.copy() 
        Te_can[j] = Te[j] * np.exp(be * np.random.uniform(-0.5, 0.5, 1)) # \tilde(Y)_i = exp(y ~ U(-0.5, 0.5)) Y_i^t-1
        L_can = Te_can/Te_can.sum() # \tilde(gamma)_i = \tilde(Y_i)/sum_j^d(\tilde(Y_j))
        tem_can = (U @ np.diag(L_can) @ np.conj(U.T)).flatten(order="F") # gamma * U * U^T (U = V in paper)
        tem = (U @ np.diag(Lamb) @ np.conj(U.T)).flatten(order="F")
        ss1 = (Pra_m @ tem_can - p_ra1)**2
        ss2 = (Pra_m @ tem - p_ra1)**2
        # ss1 = (np.repeat((Pra_m @ tem_can)[:, np.newaxis], p_ra1.shape[0], axis=1) - p_ra1[:, np.newaxis])**2
        # ss2 = (np.repeat((Pra_m @ tem)[:, np.newaxis], p_ra1.shape[0], axis=1)- p_ra1[:, np.newaxis])**2
        ss = (ss1 - ss2).sum()

In [10]:
def g():
    import jax
    import jax.random as rd
    import jax.numpy as jnp
    key = jax.random.PRNGKey(0)
    rho = jnp.zeros((d, d))
    key, subkey = rd.split(key)
    Te = rd.exponential(subkey, (d,))# np.random.standard_exponential(d) # Initial Y_i^0
    Id = jnp.eye(d)#np.eye(d)
    U = jnp.asarray(u_hat) # eigenvectors of \hat(rho) found using inversion, initial V_i^0
    Lamb = Te/Te.sum() # gamma^0
    be = 1
    p_ra1 = jnp.asarray(p_ra1)
    Pra_m = jnp.asarray(Pra).reshape((I, J))
    for i in range(10):
        for j in range(d):
            Te_can = Te.copy()
            key, subkey = rd.split(key)
            Te_can.at[j].set(Te[j] * jnp.exp(be * rd.uniform(subkey, (1,), minval=-0.5, maxval=0.5))[0]) # \tilde(Y)_i = exp(y ~ U(-0.5, 0.5)) Y_i^t-1
            L_can = Te_can/Te_can.sum() # \tilde(gamma)_i = \tilde(Y_i)/sum_j^d(\tilde(Y_j))
            tem_can = (U @ jnp.diag(L_can) @ jnp.conj(U.T)).flatten(order="F") # gamma * U * U^T (U = V in paper)
            tem = (U @ jnp.diag(Lamb) @ jnp.conj(U.T)).flatten(order="F")
            ss1 = (jnp.repeat((Pra_m @ tem_can)[:, jnp.newaxis], p_ra1.shape[0], axis=1) - p_ra1[:, jnp.newaxis])**2
            ss2 = (jnp.repeat((Pra_m @ tem)[:, jnp.newaxis], p_ra1.shape[0], axis=1)- p_ra1[:, jnp.newaxis])**2
            ss = (ss1 - ss2).sum()

In [11]:
%lprun -f g g()

UnboundLocalError: local variable 'p_ra1' referenced before assignment

## Main MH

In [7]:
%load_ext line_profiler

In [8]:
# def f():
# Main part
rho = np.zeros((d, d))
Te = np.random.standard_exponential(d) # Initial Y_i^0
Id = np.eye(d)
U = u_hat # eigenvectors of \hat(rho) found using inversion, initial V_i^0
Lamb = Te/Te.sum() # gamma^0
ro = 1/2
S = (rho_hat + np.conj(rho_hat.T))/2
be = 1

gamm = n_size/2 # lambda in paper 
entry = []
Iter = 500
burnin = 100
start_time = time.time()
Pra_m = npa(Pra).reshape((I, J))
assert False
for t in range(Iter + burnin):
    for j in range(d): # Loop for Y_i       
        Te_can = Te.copy() 
        Te_can[j] = Te[j] * np.exp(be * np.random.uniform(-0.5, 0.5, 1)) # \tilde(Y)_i = exp(y ~ U(-0.5, 0.5)) Y_i^t-1
        L_can = Te_can/Te_can.sum() # \tilde(gamma)_i = \tilde(Y_i)/sum_j^d(\tilde(Y_j))
        tem_can = (U @ np.diag(L_can) @ np.conj(U.T)).flatten(order="F") # gamma * U * U^T (U = V in paper)
        tem = (U @ np.diag(Lamb) @ np.conj(U.T)).flatten(order="F") # prev gamma * U * U^T
        #ss = (npa([tem_can.T @ x - p_ra1 for x in Pra])**2 - npa([tem.T @ x - p_ra1 for x in Pra])**2).sum() # l^prob: sum_a sum_s (Tr(v P^a_s) - hat(p^_a,s))^2
        # ss1 = (np.repeat((Pra_m @ tem_can)[:, np.newaxis], p_ra1.shape[0], axis=1) - p_ra1[:, np.newaxis])**2
        # ss2 = (np.repeat((Pra_m @ tem)[:, np.newaxis], p_ra1.shape[0], axis=1)- p_ra1[:, np.newaxis])**2
        ss1 = (Pra_m @ tem_can - p_ra1)**2
        ss2 = (Pra_m @ tem - p_ra1)**2
        ss = (ss1 - ss2).sum()
        r_prior = (ro-1) * np.log(Te_can[j]/Te[j]) - Te_can[j] + Te[j] # other part of R acceptance ratio
        ap = -gamm*np.real(ss) # other part (why use np.real?)
        if np.log(np.random.uniform(0, 1, 1)) <= ap + r_prior: Te = Te_can # if value above draw from U(0,1), then update
        Lamb = Te/Te.sum() # gamma
    for j in range(d): # Loop for V_i
        U_can = U.copy()
        U_can[:, j] = norm_complex(U[:,j] + np.random.multivariate_normal(np.zeros(d*2),np.eye(d*2)/100, size=(1)).view(np.complex128)) # Sample U/V from the unit sphere (not sure why we add to previous value)
        tem_can = (U_can @ np.diag(Lamb) @ np.conj(U_can.T)).flatten(order="F") # gamma * U * U^T
        tem = (U @ np.diag(Lamb) @ np.conj(U.T)).flatten(order="F") # gamma * U_t-1 * U^T_t-1
        # ss = (npa([tem_can.T @  x - p_ra1 for x in Pra])**2 - npa([tem.T @ x - p_ra1 for x in Pra])**2).sum() # l^prob: sum_a sum_s (Tr(v P^a_s) - hat(p^_a,s))^2
        # ss1 = (np.repeat((Pra_m @ tem_can)[:, np.newaxis], p_ra1.shape[0], axis=1) - p_ra1[:, np.newaxis])**2
        # ss2 = (np.repeat((Pra_m @ tem)[:, np.newaxis], p_ra1.shape[0], axis=1)- p_ra1[:, np.newaxis])**2
        ss1 = (Pra_m @ tem_can - p_ra1)**2
        ss2 = (Pra_m @ tem - p_ra1)**2
        ss = (ss1 - ss2).sum()
        ap = -gamm * np.real(ss) # other part of A accep ratio
        if np.log(np.random.uniform(0, 1, 1)) <= ap: U = U_can # if value above draw from U(0,1), then update

    if t > burnin:
        rho = U @ np.diag(Lamb) @ np.conj(U.T)/(t - burnin) + rho*(1-1/(t-burnin)) # approximate rho each time as rho_t = gamma_t * V_t * V_t^T /(t-burnin) + rho_t-1 / (1 - 1/(t-burnin)) -> the later we are, the more importance we give to prev rho
end_time = time.time()

AssertionError: 

In [33]:
ss1

array([5.78868868e-05-1.40232699e-20j, 1.02899535e-04-2.30959736e-20j,
       1.11638519e-05+2.86183372e-20j, ...,
       1.20184126e-05-0.00000000e+00j, 4.40831818e-05+1.15177255e-20j,
       1.13402746e-05+0.00000000e+00j])

In [41]:
%lprun -f f f()

0
mid
1
mid
2
mid
3
mid
4
*** KeyboardInterrupt exception caught in code being profiled.

Timer unit: 1e-09 s

Total time: 5.37994 s
File: /tmp/ipykernel_27398/1258079928.py
Function: f at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def f():
     2                                               # Main part
     3         1      16203.0  16203.0      0.0      rho = np.zeros((d, d))
     4         1      20603.0  20603.0      0.0      Te = np.random.standard_exponential(d) # Initial Y_i^0
     5         1      31010.0  31010.0      0.0      Id = np.eye(d)
     6         1       1886.0   1886.0      0.0      U = u_hat # eigenvectors of \hat(rho) found using inversion
     7         1      49238.0  49238.0      0.0      Lamb = Te/Te.sum() # gamma^0
     8         1       1397.0   1397.0      0.0      ro = 1/2
     9         1      31010.0  31010.0      0.0      S = (rho_hat + np.conj(rho_hat.T))/2
    10         1       1956.0   1956.0      0.0      be = 1
    11                                          

In [26]:
print(f"Took: {end_time - start_time} s")
mean_rho = np.mean((dens_ma - rho) @ np.conj((dens_ma - rho).T))
mean_rho_hat = np.mean((dens_ma - rho_hat) @ np.conj((dens_ma - rho_hat).T))
rho_evs = eig(rho)[0]

Took: 17.715636730194092 s
